## Customer Risk classifier using Random Forest

This is to demonstrate how to use random forest model to do customer risk classifier base on their demographic details, activities and balances. The sample data used here are randomly generated using Chat-GPT. 

### Load the data
First step of doing this is to load the csv of customers data into pandas dataframe. 

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Load the dataset
df = pd.read_csv('../data/others/customer_demographic_sample_10k.csv')
df.head()



,Age,Gender,Country,Job Title,Trading Volume per Month (USD),Deposit Volume per Month (USD),Withdrawal Volume per Month (USD),Current Balances (USD)
0,56,Male,Germany,Risk Manager,38125.97,8668.12,5664.24,32505.99
1,69,Female,UK,Compliance Officer,9050.05,29264.09,29563.40,95581.12
2,46,Other,USA,Portfolio Manager,11906.81,10440.46,23893.44,84692.53
3,32,Male,Brazil,Quantitative Analyst,19793.07,12596.48,23678.75,16492.17
4,60,Other,USA,Compliance Officer,15796.90,241.72,4643.74,54469.31


#### US-Sanctiond countries & High Risk Countries
define the US Sanctioned countries and High Risk countries in a list. Which will be used to map to customers country for their risk rating. 

In [6]:
# Define US-sanctioned and unstable countries for the country risk classification
sanctioned_countries = [
    'Iran', 'North Korea', 'Syria', 'Cuba', 'Russia', 'Afghanistan',
    'Balkans', 'Belarus', 'Central African Republic', 'Ethiopia', 
    'Iraq', 'Lebanon', 'Libya', 'Mali', 'Nicaragua', 'Somalia', 'South Sudan',
    'Sudan', 'Venezuela', 'Yemen', 'Zimbabwe', 'Haiti', 'Congo', 'Eritrea',
    'Cyprus', 'Burma'
]
highrisk_countries = ['China', 'Hong Kong']

#### Add a Country Risk column
Add a country risk column and map customer country to sanctioned risk and high risk rating to determine the country risk of customers.

In [7]:
# 1. Assign risk based on Country
def classify_country_risk(country):
    if country in sanctioned_countries:
        return 'Prohibited'
    elif country in highrisk_countries:
        return 'High'
    else:
        return 'Low'

df['Country Risk'] = df['Country'].apply(classify_country_risk)

df.head()

,Age,Gender,Country,Job Title,Trading Volume per Month (USD),Deposit Volume per Month (USD),Withdrawal Volume per Month (USD),Current Balances (USD),Country Risk
0,56,Male,Germany,Risk Manager,38125.97,8668.12,5664.24,32505.99,Low
1,69,Female,UK,Compliance Officer,9050.05,29264.09,29563.40,95581.12,Low
2,46,Other,USA,Portfolio Manager,11906.81,10440.46,23893.44,84692.53,Low
3,32,Male,Brazil,Quantitative Analyst,19793.07,12596.48,23678.75,16492.17,Low
4,60,Other,USA,Compliance Officer,15796.90,241.72,4643.74,54469.31,Low


#### Next Age Factor risk
Older people have more risk to fall victim to scam. Hence the risk factor are higher compare to younger customers. So any customers with age more than threshold should be set to High. 

In [8]:
# 2. Assign risk based on Age
def classify_age_risk(age):
    if age > 70:
        return 'High'
    elif 60 <= age <= 69:
        return 'Medium'
    else:
        return 'Low'

df['Age Risk'] = df['Age'].apply(classify_age_risk)
df.head()


,Age,Gender,Country,Job Title,Trading Volume per Month (USD),Deposit Volume per Month (USD),Withdrawal Volume per Month (USD),Current Balances (USD),Country Risk,Age Risk
0,56,Male,Germany,Risk Manager,38125.97,8668.12,5664.24,32505.99,Low,Low
1,69,Female,UK,Compliance Officer,9050.05,29264.09,29563.40,95581.12,Low,Medium
2,46,Other,USA,Portfolio Manager,11906.81,10440.46,23893.44,84692.53,Low,Low
3,32,Male,Brazil,Quantitative Analyst,19793.07,12596.48,23678.75,16492.17,Low,Low
4,60,Other,USA,Compliance Officer,15796.90,241.72,4643.74,54469.31,Low,Medium


In [9]:
# 3. Assign risk based on Job Title
political_jobs = ['Politician', 'Government Official']
def classify_job_risk(job_title):
    if job_title in political_jobs:
        return 'High'
    else:
        return 'Low'

df['Job Risk'] = df['Job Title'].apply(classify_job_risk)
df.head()



,Age,Gender,Country,Job Title,Trading Volume per Month (USD),Deposit Volume per Month (USD),Withdrawal Volume per Month (USD),Current Balances (USD),Country Risk,Age Risk,Job Risk
0,56,Male,Germany,Risk Manager,38125.97,8668.12,5664.24,32505.99,Low,Low,Low
1,69,Female,UK,Compliance Officer,9050.05,29264.09,29563.40,95581.12,Low,Medium,Low
2,46,Other,USA,Portfolio Manager,11906.81,10440.46,23893.44,84692.53,Low,Low,Low
3,32,Male,Brazil,Quantitative Analyst,19793.07,12596.48,23678.75,16492.17,Low,Low,Low
4,60,Other,USA,Compliance Officer,15796.90,241.72,4643.74,54469.31,Low,Medium,Low


In [11]:
# Combine the risk levels into a final 'Risk Level' column
def combine_risk_levels(row):
    if row['Country Risk'] == 'Prohibited':
        return 'Prohibited'
    elif row['Country Risk'] == 'High' or row['Age Risk'] == 'High' or row['Job Risk'] == 'High':
        return 'High'
    elif row['Age Risk'] == 'Medium':
        return 'Medium'
    else:
        return 'Low'

df['Risk Level'] = df.apply(combine_risk_levels, axis=1)
df.head()


,Age,Gender,Country,Job Title,Trading Volume per Month (USD),Deposit Volume per Month (USD),Withdrawal Volume per Month (USD),Current Balances (USD),Country Risk,Age Risk,Job Risk,Risk Level
0,56,Male,Germany,Risk Manager,38125.97,8668.12,5664.24,32505.99,Low,Low,Low,Low
1,69,Female,UK,Compliance Officer,9050.05,29264.09,29563.40,95581.12,Low,Medium,Low,Medium
2,46,Other,USA,Portfolio Manager,11906.81,10440.46,23893.44,84692.53,Low,Low,Low,Low
3,32,Male,Brazil,Quantitative Analyst,19793.07,12596.48,23678.75,16492.17,Low,Low,Low,Low
4,60,Other,USA,Compliance Officer,15796.90,241.72,4643.74,54469.31,Low,Medium,Low,Medium


In [12]:
# Preprocess categorical features using LabelEncoder
label_encoder = LabelEncoder()

df['Gender'] = label_encoder.fit_transform(df['Gender'])
df['Country'] = label_encoder.fit_transform(df['Country'])
df['Job Title'] = label_encoder.fit_transform(df['Job Title'])

# Select features and the target
X = df[['Age', 'Gender', 'Country', 'Job Title', 
        'Trading Volume per Month (USD)', 
        'Deposit Volume per Month (USD)', 
        'Withdrawal Volume per Month (USD)', 
        'Current Balances (USD)']]

y = df['Risk Level']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predict and evaluate the model
y_pred = rf_model.predict(X_test)

# Print classification report and accuracy score
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))




              precision    recall  f1-score   support

        High       1.00      1.00      1.00       744
         Low       1.00      1.00      1.00      1808
      Medium       1.00      1.00      1.00       448

    accuracy                           1.00      3000
   macro avg       1.00      1.00      1.00      3000
weighted avg       1.00      1.00      1.00      3000

Accuracy: 0.9996666666666667
